In [ ]:
# default_exp prep

# Prep

> This module contains all the necessary functions for loading and preprocessing videos into a standard format to be used in the rest of the pipeline.

In [ ]:
#export
import cv2
import ffmpeg
import numpy
import random
import scenedetect

# scenedetect
from scenedetect.video_manager import VideoManager
from scenedetect.scene_manager import SceneManager
from scenedetect.frame_timecode import FrameTimecode
from scenedetect.stats_manager import StatsManager
from scenedetect.detectors import ContentDetector
from scenedetect.video_splitter import is_ffmpeg_available, split_video_ffmpeg

from pathlib import Path



In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def fix_frame_rate(video_paths, fr = 30):
    """
        Fixes each video in the list of video paths to a certain frame rate
        
        Returns a list of paths to the new fixed frame rate videos
    """
    fixed_paths = []
    for vp in video_paths:
        try:
            stream = ffmpeg.input(vp)
            stream = ffmpeg.output(stream, str(vp.parent/"fixed.mp4"), r = fr)
            stream = ffmpeg.overwrite_output(stream)
            out, err = ffmpeg.run(stream)
            
            fixed_paths.append(vp.parent/"fixed.mp4")
        except Exception as e:
            print("Error occured:", e)
    
    return fixed_paths

In [ ]:
path = Path("/tf/data/datasets/videos")
video_paths = sorted(path.glob("**/video.mp4")); video_paths[:6]

[PosixPath('/tf/data/datasets/videos/art_and_design/ibis_paint/scenario1/video.mp4'),
 PosixPath('/tf/data/datasets/videos/art_and_design/ibis_paint/scenario2/video.mp4'),
 PosixPath('/tf/data/datasets/videos/art_and_design/pixel_art_paint/scenario1/video.mp4'),
 PosixPath('/tf/data/datasets/videos/art_and_design/pixel_art_paint/scenario2/video.mp4'),
 PosixPath('/tf/data/datasets/videos/auto_and_vehicles/car_part/scenario1/video.mp4'),
 PosixPath('/tf/data/datasets/videos/auto_and_vehicles/car_part/scenario2/video.mp4')]

In [ ]:
fr = 30.0
fixed_paths = fix_frame_rate(video_paths[4:6], fr)

In [ ]:
for fp in fixed_paths[:2]:
    assert fr == eval(ffmpeg.probe(fp)["streams"][0]["avg_frame_rate"])

In [ ]:
#export
def chunk_videos(vid_paths):
    """Chunks videos into different scenes based on their content for later processing."""
    
    for vp in vid_paths:
        try:
            # Setup the different managers for chunking the scenes.
            video_manager = VideoManager([str(vp)])
            stats_manager = StatsManager()
            scene_manager = SceneManager(stats_manager)

            # Add ContentDetector algorithm (constructor takes detector options like threshold).
            scene_manager.add_detector(ContentDetector())
            base_timecode = video_manager.get_base_timecode()

            # Set downscale factor to improve processing speed (no args means default).
            video_manager.set_downscale_factor()

            # Set the duration to be however long the video is and start the video manager.
            video_manager.set_duration()
            video_manager.start()

            # Perform scene detection on video_manager and grab the scenes.
            scene_manager.detect_scenes(frame_source=video_manager)
            scene_list = scene_manager.get_scene_list(base_timecode)

            # If the output dir of the chunked videos does not exist, create it.
            if not (vp.parent/"chunks").exists():
                (vp.parent/"chunks").mkdir()
                
            # Split the video into chunks based on the scene list and save to the "chunks" folder.
            split_video_ffmpeg([vp], scene_list,
                               str(vp.parent/"chunks/$VIDEO_NAME-$SCENE_NUMBER.mp4"),
                               "chunk"#, arg_override = args
                              )
            with open("stats.csv", 'w') as stats_file:
                stats_manager.save_to_csv(stats_file, base_timecode)
        finally:
            # Close out the video_manager.
            video_manager.release()

In [ ]:
chunk_videos(fixed_paths[:1])





  0%|          | 0/1027 [00:00<?, ?frames/s]



  1%|          | 7/1027 [00:00<00:15, 65.97frames/s]



  2%|▏         | 17/1027 [00:00<00:13, 72.33frames/s]



  3%|▎         | 27/1027 [00:00<00:12, 78.51frames/s]



  4%|▎         | 37/1027 [00:00<00:11, 83.65frames/s]



  5%|▍         | 47/1027 [00:00<00:11, 87.14frames/s]



  6%|▌         | 57/1027 [00:00<00:10, 88.49frames/s]



  7%|▋         | 67/1027 [00:00<00:10, 90.05frames/s]



  7%|▋         | 77/1027 [00:00<00:10, 91.49frames/s]



  8%|▊         | 87/1027 [00:00<00:10, 93.22frames/s]



  9%|▉         | 97/1027 [00:01<00:10, 92.98frames/s]



 10%|█         | 107/1027 [00:01<00:09, 93.95frames/s]



 11%|█▏        | 117/1027 [00:01<00:09, 94.91frames/s]



 12%|█▏        | 127/1027 [00:01<00:09, 93.82frames/s]



 13%|█▎        | 137/1027 [00:01<00:09, 94.19frames/s]



 14%|█▍        | 147/1027 [00:01<00:09, 95.56frames/s]



 15%|█▌        | 157/1027 [00:01<00:09, 96.29frames/s]



 16%|█▋        | 167/1027 [00:0


Writing 1026 frames to CSV...


In [ ]:
stream = ffmpeg.input(video_paths[4])

In [ ]:
stream = ffmpeg.filter(stream, "decimate")

In [ ]:
stream = ffmpeg.output(stream, 'output.mp4')

In [ ]:
out, err = ffmpeg.run(stream)

In [ ]:
#export
def get_rand_imgs(vid_path, max_msecs, n = 10):
    vid = cv2.VideoCapture(str(vid_path))
    
    imgs = []
    while len(imgs) < n:
        msec = random.randrange(1_000, max_msecs, 1_000)
        vid.set(cv2.CAP_PROP_POS_MSEC, msec)
        
        success, img = vid.read()
        if success:
            imgs.append(img)
    
    return imgs

In [ ]:
imgs = get_rand_imgs(video_paths[0], 30_000, n = 5)

In [ ]:
for img in imgs:
    plt.imshow(img)
    plt.show()

In [ ]:
#export
def vid_from_frames(frames):
    """Generate video from list of frame paths."""
    

In [ ]:
test_path = Path("/tf/data/datasets/videos")

In [ ]:
stream = ffmpeg.input(video_paths[4])

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_prep.ipynb.
Converted 01_features.ipynb.
Converted index.ipynb.
Converted lesson1-pets.ipynb.
